In [ ]:
# making the imports
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from functools import reduce
import numpy as np
from itertools import permutations, product
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from scipy.sparse import csr_matrix
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import math
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def beautify_plot(ttl, xl, yl, lg=True):
    '''
    Overrides the default title, xlabel, ylabel and legends
    '''
    plt.title(ttl)
    plt.xlabel(xl)
    plt.ylabel(yl);
    if lg:
        plt.legend();
        
base_loc = '/kaggle/input/restaurant-data-with-consumer-ratings/'

In [ ]:
df_master_train = [pd.read_csv(base_loc+'rating_final.csv')] # read the data
df_master_test = [] # prepare the list to reduce the compiled profiling data
imputation_rules = {} # store rules for imputation

df_master_train[0].head() # get the head

In [ ]:
df_master_train[0].placeID.nunique(), df_master_train[0].userID.nunique() # get the unique users and restaurants

In [ ]:
def get_info(df_temp, df_type):
    '''
    Function to get the customized details of a dataframe.
    '''
    print(f'The shape is :: {df_temp.shape}\n\nThe # duplicates :: {df_temp.duplicated().sum()}')
    print(f'\nThe # unique per column ::\n{df_temp.nunique()}')
    print(f'\nThe # NaNs per column ::\n{df_temp.isna().sum()}')
    if df_type=='chef':
        print(f'\nThe % Primary Key common in ratings and this df :: {pd.merge(df_temp, df_master_train[0]).placeID.nunique()/130*100:.2f}%')
    if df_type=='user':
        print(f'\nThe % Primary Key common in ratings and this df :: {pd.merge(df_temp, df_master_train[0]).userID.nunique()/138*100:.2f}%')

In [ ]:
def process_categorical(values):
    '''
    Cleans and normalize categorical variable that has text
    '''
    return [i.strip().lower().replace(' ','_',).replace('-','_') for i in values]

We see that we have need to predict the `rating`. Now this is very obvious that all the restaurants will not be there in the ratings dataframe. Again, ideally all the customers should be.

So, let's first try to see how the restaurant details fit in for the ratings dataset.
 
`chef*` datasets are for the restaurants. Let's try to see what they have one by one and make them compatible for preparation of the final dataset.

### 1.1 chefmozaccepts

In [ ]:
df1 = pd.read_csv(base_loc+'chefmozaccepts.csv') # read and get info on the data
get_info(df1, 'chef')

In [ ]:
# plot the Count of Accepted Payment Types across all Restaurants
df1.Rpayment.value_counts().plot(kind='bar')
beautify_plot('Count of Accepted Payment Types across all Restaurants\n', 'Payment Types', 'Count')

A cool observation is that most of the places accepts cash. One more thing to notice is that 'VISA' and 'visa' are different values and it seems that it may be just an error. So, we process the categorical field.

In [ ]:
df1['Rpayment'] = process_categorical(df1.Rpayment) # clean the Rpayment column

In [ ]:
# plot the Distribution of Restautants accepting unique number of Payemnt Types
df1.groupby('placeID').count().sort_values('Rpayment').plot(kind='hist', bins=7);
beautify_plot('Distribution of Restautants accepting unique number of Payemnt Types\n', 
              '# Unique Accepted Payment Types', 
              'Frequency')

A quick glance reveals that most of the places accepts a single mode of payment.

In [ ]:
# plot the Count of various Payment Types in Restautants which accepts different unique mode(s) of Payment
df11 = df1.groupby('placeID').count().reset_index()
df11.columns = ['placeID', 'count']
df12 = df1.merge(df11).query('count<4').groupby('count').Rpayment.value_counts().unstack()
for i in range(0,df12.shape[0]):
    plt.subplots(1)
    df12.iloc[i].plot.bar()
    beautify_plot(f'Count of various Payment Types in Restautants which accepts only {i+1} unique mode(s) of Payment\n', 
                  'Accepted Payment Types', 
                  'Count')

We see the observation for those stores which accepts 1,2 and 3 mode of payments.
* We see that those most of the stores who accepts only 1 mode of payment, accepts cash.
* We see that those most of the stores who accepts only 2 modes of payment, accepts mostly visa, cash and debit cards.
* We see that those most of the stores who accepts only 3 mode of payment, accepts mostly visa, mastercards and less cash and lesser debit cards.

This may reveal a pattern that those stores which accept single mode of payments, does attract people with `low budgets` and those with more attracts people with `high or medium` budget.

We see that after the merge, there will be some restaurants who will have NaNs for the card types. We can impute them with `0`s. 

Also we see that are around 12 levels. So, we can conclude that we can accept them for the master data after One Hot Encoding.

In [ ]:
df1_final = pd.get_dummies(df1).groupby('placeID').sum().reset_index() # one hot encode the features

# add the dataframe to the training and testing list
df_master_train.append(df1_final)
df_master_test.append(df1_final)

# store the imputation rules
imputation_rules.update({i:{'fillna': 0} for i in df1_final.columns[1:]})

In [ ]:
df2 = pd.read_csv(base_loc+'chefmozcuisine.csv') # read and get info on the data
get_info(df2, 'chef')

In [ ]:
# plot the Count of Cuisines across all Restaurants
df2['Rcuisine'] = process_categorical(df2.Rcuisine)
cusines = df2.Rcuisine.value_counts()[:10] # get the top 10 cusines
cusines.plot.barh()
beautify_plot('Count of Cuisines across all Restaurants\n', 'Counts', 'Cuisines');

In [ ]:
# bin the cuisines which are not in top 10 as others
df2['Rcuisine'] = df2.Rcuisine.replace([i for i in df2.Rcuisine.unique() if i not in cusines], 'others')

In [ ]:
# plot the Counts of various Cusines offered by Restaurants where only single mode of Payment is accepted
df1.groupby('placeID').count().query('Rpayment==1').reset_index().merge(df2).Rcuisine.value_counts().plot.bar()
beautify_plot('Counts of various Cusines offered by Restaurants where only single mode of Payment is accepted\n',
              'Cusines', 'Count');

In [ ]:
# plot the Counts of various Cusines offered by Restaurants where more than one mode of Payment is accepted
df1.groupby('placeID').count().query('Rpayment>1').reset_index().merge(df2).Rcuisine.value_counts().plot.bar()
beautify_plot('Counts of various Cusines offered by Restaurants where more than one mode of Payment is accepted\n',
              'Cusines', 'Count');

We do get quite an interesting pattern here. Previously we saw that **single mode of payments has predominant mode of payment as cash** and again, we see that those restaurants offer mostly **Mexican, American Cuisines and Bars**.

Also, for two or more mode of payments, there are **cards involved** and as per the distribution of the cuisines, we see that apart from Mexican and American (which can be burger joints), we can see more delicate **European cuisines like International, French, Italian etc**.

Once we have processed this dataframe, we will account for the NaNs after the join by imputing `1 in others` and `0 in rest of the cuisines`.  

In [ ]:
df2_final = pd.get_dummies(df2).groupby('placeID').sum().reset_index() # one hot encode the features

# add the dataframe to the training and testing list
df_master_train.append(df2_final)
df_master_test.append(df2_final)

# store the imputation rules
imputation_rules.update({i: {'fillna': 1} if i=='Rcuisine_others' else {'fillna':0} for i in df2_final.columns[1:]})

### 1.3 chefmozhours4

In [ ]:
df3 = pd.read_csv(base_loc+'chefmozhours4.csv') # read and get info on the data
get_info(df3, 'chef')

df3.head()

In [ ]:
# plot the Count of days open across all Restaurants
df3.groupby('placeID').agg({'days': pd.Series.nunique}).days.value_counts().plot.bar();
beautify_plot('Count of days open across all Restaurants\n', 'Days Open', 'Count');

The data ideally should have `n*3 rows`, i.e the opening and closing hours for each place for Weekdays, Saturdays and Sundays. However we have some as 1 and 2 also. That means, if the shop is not open in either of the weekdays, sat or sun, it does not have a record.


We encounter a number of observations here.
* Duplicates are present
* The hours column has 273 unique values

We propose to do the following for the processing.

* Convert the hours column to number of actual numerical hours. If the hours come as 0, we say the shop is open for 24 hours. This is based on the previous observation of the distribution of days.
* Remove the duplicates on place and days (cannot have 2 records with different opening and closing timings for the same day)
* Do a pivot on the days and account the dataset for inclusion in the master file. Do a fillna of `0` for days in which it was not open.

The rule for imputation when creating the master will be impute by mode.

In [ ]:
df3 = pd.read_csv(base_loc+'chefmozhours4.csv')
# feature engineer the actual hours
df3['hrs'] = df3.hours.apply(lambda x: np.ptp([int(j[:2]) for i in x.split(';') for j in i.split('-') if j]))
df3 = df3.replace(0, 24) # replace 0 (came as diff) with 24 for shops which are open round the clock

df3.drop_duplicates(subset=['placeID', 'days'], inplace=True)

# one hot encode the features
df3 = df3.pivot(index='placeID', columns='days', values='hrs')\
       .reset_index().rename_axis(None, axis=1)\
       .fillna(0).astype(int) # replace 0 for shops which are not open on days (after pivot)

# add the dataframe to the training and testing list
df3.columns = ['placeID', 'mon_tue_wed_thu_fri', 'sat', 'sun']
df_master_train.append(df3)
df_master_test.append(df3)

# store the imputation rules
imputation_rules.update({i: 'mode' for i in df3.columns[1:]})

### 1.4 chefmozparking

In [ ]:
df4 = pd.read_csv(base_loc+'chefmozparking.csv') # read and get info on the data
get_info(df4, 'chef')

In [ ]:
# plot the Counts of Parking Lots across all Restuatrants
df4.parking_lot.value_counts().plot.barh();
beautify_plot('Counts of Parking Lots across all Restuatrants\n', 'Parking Lot Types', 'Count');

The above data looks fine and is incorporated. We will create the rule for imputation with mode for any NaNs.

In [ ]:
df4_final = pd.get_dummies(df4).groupby('placeID').sum().reset_index() # one hot encode the features

# add the dataframe to the training and testing list
df_master_train.append(df4_final)
df_master_test.append(df4_final)

# store the imputation rules
imputation_rules.update({i: 'mode' for i in df4_final.columns[1:]})

### 1.5 usercuisine

In [ ]:
df5 = pd.read_csv(base_loc+'usercuisine.csv') # read and get info on the data
get_info(df5, 'user')

In [ ]:
# plot the Counts of various Cusines preferred by all the Customers
df5['Ucuisine'] = process_categorical(df5.Rcuisine) # get the top 10 cusines
df5.Ucuisine.value_counts()[:10].plot.bar();
beautify_plot('Counts of various Cusines preferred by all the Customers\n', 'Cusines', 'Count');

We again need to bin the less popular ones in `others`.

Again we create the rule of replacing NaNs with others as 1 for User cuisines.

In [ ]:
# bin the cuisines which are not in top 10 as others
df5['Ucuisine'] = df5.Ucuisine.replace([i for i in df5.Ucuisine.unique() if i not in cusines], 'others')
# one hot encode the features
df5_final = pd.concat([df5[['userID']], pd.get_dummies(df5.Ucuisine, prefix='Ucuisine')], axis=1)\
              .groupby('userID').sum().reset_index()

# add the dataframe to the training and testing list
df_master_train.append(df5_final)
df_master_test.append(df5_final)

# store the imputation rules
imputation_rules.update({i: {'fillna': 1} if i=='Ucuisine_others' else {'fillna':0} for i in df5_final.columns[1:]})

### 1.6 userpayment

In [ ]:
df6 = pd.read_csv(base_loc+'userpayment.csv') # read and get info on the data
df6['Upayment'] = process_categorical(df6.Upayment)
get_info(df6, 'user')

In [ ]:
# plot the Counts of Payment Methods preferred by all the Customers
df6.Upayment.value_counts().plot(kind='bar');
beautify_plot('Counts of Payment Methods preferred by all the Customers\n', 'Payment Methods', 'Count');

In [ ]:
# plot the Counts of Payment Methods preferred by Customers who use various mode(s) of Payment(s)
df61 = df6.groupby('userID').count().reset_index()
df61.columns = ['userID', 'count']
df62 = df6.merge(df61).groupby('count').Upayment.value_counts().unstack()
for i in range(0,df62.shape[0]):
    plt.subplots(1)
    df62.iloc[i].plot.bar()
    beautify_plot(f'Counts of Payment Methods preferred by Customers who use {i+1} mode(s) of Payment(s)\n', 'Payment Methods', 'Count');

* The distribution that we see here is that the Users who like to pay via a single mode, it is always `cash`.
* For the others, they almost equally like to pay via cash, and cards.

For the customer who don't have this data, the imputation will happen by adding `1` to cash.

In [ ]:
df6_final = pd.concat([df6[['userID']], pd.get_dummies(df6.Upayment, prefix='Upayment')], axis=1)\
              .groupby('userID').sum().reset_index() # one hot encode the features

# add the dataframe to the training and testing list
df_master_train.append(df6_final)
df_master_test.append(df6_final)

# store the imputation rules
imputation_rules.update({i:{'fillna': 1} if i=='Upayment_cash' else {'fillna': 0} for i in df6_final.columns[1:]})

### 1.7 userprofile

In [ ]:
df7 = pd.read_csv(base_loc+'userprofile.csv') # read and get info on the data
get_info(df7, 'user')

In [ ]:
df7['age'] = (2020-df7.birth_year) # get the age

# get rid of some of the useless columns
to_drop = ['latitude', 'longitude', 'height', 'birth_year']
df7 = df7.drop(to_drop, axis=1)

In [ ]:
# Count of all the categorical variables
df7 = df7.replace('?', 'none') # replace the ? with none
for i in list(df7.select_dtypes('object').columns)[1:]:
    df7[i] = process_categorical(df7[i])
    plt.subplots(1)
    df7[i].value_counts().plot.bar()
    beautify_plot(f'Count of {i}(s) among all Users\n', f'{i}(s)', 'Count');

Among the various relations, we can see that the budget after grouping together the `low`, `none` and `high`, `medium` can be a nice way to segment the data for the models and build two different ones.

In [ ]:
df7['budget'] = df7.budget.replace('none', 'low').replace('high', 'medium') # bin the budget

In [ ]:
# plot the Histogram of Weight of all Users\
df7.weight.plot.hist();
beautify_plot(f'Histogram of Weight of all Users\n', 'Frequency', 'Weight');

In [ ]:
# plot the Histogram of Age of all Users\
df7.age.plot.hist();
beautify_plot(f'Histogram of Age of all Users\n', 'Frequency', 'Age');

In [ ]:
df7_final = pd.concat([pd.get_dummies(df7.drop(['userID', 'age', 'weight'], axis=1)), df7[['userID', 'age', 'weight']]], axis=1)\
              .groupby('userID').sum().reset_index() # one hot encode the features

# add the dataframe to the training and testing list
df_master_train.append(df7_final) 
df_master_test.append(df7_final)

## 2. Creating Training Data

In [ ]:
# reduce and create a single dataframe
df_train = reduce(lambda x,y: x.merge(y, 'left'), df_master_train)
assert(df_train.shape[0] == df_master_train[0].shape[0])

In [ ]:
# apply the imputation rules
print(f'# NaNs :: {reduce(lambda x,y: x+y, df_train.isna().sum())}')
for i in imputation_rules.keys():
    if type(imputation_rules[i]) == dict: # if it is fillna
        for j in imputation_rules[i].keys():
            if j == 'fillna':
                df_train[i] = df_train[i].fillna(imputation_rules[i][j])
    else: # if it is mode
        df_train[i] = df_train[i].fillna(df_train[i].mode()[0])
print(f'# NaNs after applying Rules :: {reduce(lambda x,y: x+y, df_train.isna().sum())}')

## 3. Creating Test and Validation Data

In [ ]:
# create exhaustive list of places and users which appear in the files
places = set(reduce(lambda x,y: x[['placeID']].append(y[['placeID']]), df_master_test[:4]).placeID)
users = set(reduce(lambda x,y: x[['userID']].append(y[['userID']]), df_master_test[4:]).userID)
df_test = pd.DataFrame(list(product(places, users)), columns=['placeID', 'userID'])

# take only those which are not rated
df_test = df_test.merge(df_train[['placeID', 'userID']], 'outer', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
ast_shape = df_test.shape

# get the profiles for this master data
df_test = [df_test] + df_master_test
df_test = reduce(lambda x,y: x.merge(y, 'left'), df_test)

assert(df_test.shape[0] == ast_shape[0])

In [ ]:
# apply the imputation rules
print(f'# NaNs :: {reduce(lambda x,y: x+y, df_test.isna().sum())}')
for i in imputation_rules:
    if type(imputation_rules[i]) == dict: # if fillna
        for j in imputation_rules[i].keys():
            if j == 'fillna':
                df_test[i] = df_test[i].fillna(imputation_rules[i][j])
    else: # if mode
        df_test[i] = df_test[i].fillna(df_test[i].mode()[0])
print(f'# NaNs after applying Rules :: {reduce(lambda x,y: x+y, df_test.isna().sum())}')

In [ ]:
# plot the Overall Rating Counts
df_train.rating.value_counts().plot.bar()
beautify_plot('Overall Rating Counts\n', 'Ratings', 'Counts');

We see no imbalance in the dataset or presence of outliers in this case. We need to convert the ratings into a continuous value. We will add some noise (0-1 in real line) to the rating and do a log transform.

In [ ]:
# create the validation training and test data
to_drop = ['rating', 'service_rating', 'food_rating', 'userID', 'placeID']
X_train = df_train.drop(to_drop, axis=1).astype('int')
y_train = df_train.rating

X_vtrain, X_vtest, y_vtrain, y_vtest = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

y_vtrain_cont = np.log1p(y_vtrain+np.random.rand())
y_vtest_cont = np.log1p(y_vtest+np.random.rand())

## 4. Regression Models
### 4.1 Linear Regression Model

In [ ]:
def percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred)*0.50))

In [ ]:
res = sm.OLS(y_vtrain_cont, X_vtrain).fit()
res.summary()

In [ ]:
y_train_pred = list(map(math.floor, np.e**res.predict(X_vtrain)))
print('Training Accuracy and Error')
print('---------------------------')
print(f'Accuracy :: {accuracy_score(y_vtrain, y_train_pred):.2f}\nError :: {percentage_error(y_vtrain, y_train_pred):.2f}')

In [ ]:
print('Test Accuracy and Error')
print('------------------------')
y_pred = list(map(math.floor, np.e**res.predict(X_vtest)))
print(f'Accuracy :: {accuracy_score(y_vtest, y_pred):.2f}\nError :: {percentage_error(y_vtest, y_pred):.2f}')

We can see that the linear model didn't have a very good adjusted R2. But it's still acceptable, given the sparse nature.

When the accuracy score when the scores were converted back to the classes, the manual % error indicates that in the misclassified ones, it on an average made a mistake of saying +-1 of the actual rating.

On the test data, the error is 0.55 which means that the model mostly made much mistake to give the ratings ~(+-1 of the actual rating)

### 4.2 Decision Tree Regression Model

In [ ]:
dt_res = DecisionTreeRegressor().fit(X_vtrain, y_vtrain_cont)

In [ ]:
y_train_pred = list(map(math.floor, np.e**dt_res.predict(X_vtrain)))
print('Training Accuracy and Error')
print('---------------------------')
print(f'Accuracy :: {accuracy_score(y_vtrain, y_train_pred):.2f}\nError :: {percentage_error(y_vtrain, y_train_pred):.2f}')

In [ ]:
print('Test Accuracy and Error')
print('------------------------')
y_pred = list(map(math.floor, np.e**dt_res.predict(X_vtest)))
print(f'Accuracy :: {accuracy_score(y_vtest, y_pred):.2f}\nError :: {percentage_error(y_vtest, y_pred):.2f}')

## 5. Classification Models
### 5.1 Logistic regression

In [ ]:
lr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=5000)
lr.fit(X_vtrain, y_vtrain)

In [ ]:
y_train_pred = list(map(math.floor, lr.predict(X_vtrain)))
print('Training Accuracy and Error')
print('---------------------------')
print(f'Accuracy :: {accuracy_score(y_vtrain, y_train_pred):.2f}\nError :: {percentage_error(y_vtrain, y_train_pred):.2f}')

In [ ]:
print('Test Accuracy and Error')
print('------------------------')
y_pred = list(map(math.floor, lr.predict(X_vtest)))
print(f'Accuracy :: {accuracy_score(y_vtest, y_pred):.2f}\nError :: {percentage_error(y_vtest, y_pred):.2f}')

### 5.2 Naive Bayes

In [ ]:
nb = MultinomialNB()
nb.fit(X_vtrain, y_vtrain)

In [ ]:
y_train_pred = list(map(math.floor, nb.predict(X_vtrain)))
print('Training Accuracy and Error')
print('---------------------------')
print(f'Accuracy :: {accuracy_score(y_vtrain, y_train_pred):.2f}\nError :: {percentage_error(y_vtrain, y_train_pred):.2f}')

In [ ]:
print('Test Accuracy and Error')
print('------------------------')
y_pred = list(map(math.floor, nb.predict(X_vtest)))
print(f'Accuracy :: {accuracy_score(y_vtest, y_pred):.2f}\nError :: {percentage_error(y_vtest, y_pred):.2f}')

### 5.3 Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=50)
rf.fit(X_vtrain, y_vtrain)

In [ ]:
y_train_pred = list(map(math.floor, rf.predict(X_vtrain)))
print('Training Accuracy and Error')
print('---------------------------')
print(f'Accuracy :: {accuracy_score(y_vtrain, y_train_pred):.2f}\nError :: {percentage_error(y_vtrain, y_train_pred):.2f}')

In [ ]:
print('Test Accuracy and Error')
print('------------------------')
y_pred = list(map(math.floor, rf.predict(X_vtest)))
print(f'Accuracy :: {accuracy_score(y_vtest, y_pred):.2f}\nError :: {percentage_error(y_vtest, y_pred):.2f}')

### 5.4 SVM

In [ ]:
svc = SVC(gamma='auto')
svc.fit(X_vtrain, y_vtrain)

In [ ]:
y_train_pred = list(map(math.floor, svc.predict(X_vtrain)))
print('Training Accuracy and Error')
print('---------------------------')
print(f'Accuracy :: {accuracy_score(y_vtrain, y_train_pred):.2f}\nError :: {percentage_error(y_vtrain, y_train_pred):.2f}')

In [ ]:
print('Test Accuracy and Error')
print('------------------------')
y_pred = list(map(math.floor, svc.predict(X_vtest)))
print(f'Accuracy :: {accuracy_score(y_vtest, y_pred):.2f}\nError :: {percentage_error(y_vtest, y_pred):.2f}')

## 6. Comparison of the Models

In [ ]:
# store the metrics
evaluataion = {
     'Model': {0: 'Linear Regression',
               1: 'Decision Tree Regressor',
               2: 'Logistic Regression',
               3: 'Naive Bayes',
               4: 'Random Forest',
               5: 'SVM'},
     'Testing_Accuracy': {0: 0.21, 1: 0.22, 2: 0.53, 3: 0.52, 4: 0.6, 5: 0.46},
     'Testing_Error': {0: 0.55, 1: 0.51, 2: 0.29, 3: 0.29, 4: 0.24, 5: 0.32},
     'Training_Accuracy': {0: 0.17, 1: 0.0, 2: 0.63, 3: 0.48, 4: 0.99, 5: 0.63},
     'Training_Error': {0: 0.58, 1: 0.51, 2: 0.23, 3: 0.31, 4: 0.01, 5: 0.23},
     'Type': {0: 'Regression',
              1: 'Regression',
              2: 'Classification',
              3: 'Classification',
              4: 'Classification',
              5: 'Classification'}}
evaluataion = pd.DataFrame(evaluataion)
evaluataion.groupby('Type').mean().plot.bar();

We can see that the Testing Error went down for the classification models on an average by 50% and hence, we would proceed with the classification models.

In [ ]:
evaluataion.query('Type=="Classification"')

## 7. Predictions and Recommendation

In [ ]:
# make the recommendations
y_test = pd.DataFrame(nb.predict(df_test.drop(['placeID', 'userID'], axis=1)), columns=['predicted_rating'])
df_recommend = pd.concat([df_test[['placeID', 'userID']], y_test], axis=1) # create exhaustive list

# get the top n recommendations and save it in a file
n = 3
df_recommend = df_recommend.sort_values(['userID', 'predicted_rating'], ascending=False).groupby('userID').head(n)
df_recommend.to_csv('final_recommendations.csv', index=False)